# Final Exam Solution

## FINM 37500 - 2024

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

Thanks to
* Younghun Lee
* Jordan Sacks
* Burak Sekerci

***

# Instructions

## Please note the following:

Points
* You have **180 minutes** to complete the exam.
* For every minute late you submit the exam, you will lose one point.
Final Exam

Submission
* You will upload your solution to the Final Exam assignment on Canvas, where you downloaded this.
* Your submission should be readable, (the graders can understand your answers,) and it should include all code used in your analysis in a file format that the code can be executed. (ie. .ipynb preferred, .pdf is unacceptable.)

Rules
* The exam is open-material, closed-communication.
* You do not need to cite material from the course github repo--you are welcome to use the code posted there without citation, (only for this exam.)

Advice
* If you find any question to be unclear, state your interpretation and proceed. We will only answer questions of interpretation if there is a typo, error, etc.
* The exam will be graded for partial credit.

Answer Quality
* For conceptual questions, note that we will grade your answer for its relevance to our course focus and discussion. 
* Making points that are irrelevant, out-of-context, or overly general will may not help and could hurt your score for the question.
* This may be particularly relevant for answers which are copied from LLMs such as ChatGPT, but do not get at the heart of our contextual course learning.

## Scoring

| Problem | Points |
|---------|--------|
| 1       | 20     |
| 2       | 25     |
| 3       | 50     |
| 4       | 55     |
| **Total**   | **150**|

#### Each numbered question is worth 5pts unless otherwise noted.

***

## Data

**All data files are found in the class github repo, in the `data` folder.**

This exam makes use of the following data files:
* `exam_data_2024-03-05.xlsx`

This file has sheets for...
* curve data - discount factors and forward volatilities
* BDT tree of rates (continuously compounded, as usual)
* vol quotes (across strikes) on swaptions for a particular expiry and tenor
* SABR parameters for a vol skew of the expiry and tenor associted with the vol quotes on the previous sheet.

Note
* the curve data is given at quarterly frequency
* all rates reported in the curve data are quarterly compounded, which is conveneint for the study of quarterly products in this exam (whether caps, floors, swaps, etc.)

### Load Data

If useful, the following code loads the data:

In [1]:
import numpy as np
import pandas as pd

In [2]:
DATE = '2024-03-05'

FILEIN = f'../data/exam_data_{DATE}.xlsx'

sheet_curves = 'rate curves'
sheet_tree = 'rate tree'
sheet_volskew = 'bloomberg vcub'
sheet_sabrparams = 'sabr params'

curves = pd.read_excel(FILEIN, sheet_name=sheet_curves).set_index('tenor')

ratetree = pd.read_excel(FILEIN, sheet_name=sheet_tree).set_index('state')
ratetree.columns.name = 'time'

volskew = pd.read_excel(FILEIN, sheet_name=sheet_volskew)

sabrparams = pd.read_excel(FILEIN, sheet_name=sheet_sabrparams).set_index('parameter')

***

# 1. Models

## 1.1. (10pts)

Be specific as to which type of data is used to estimate a binomial rate tree. Or, if the feature is assumed rather than estimated, make that clear.

How do we estimate the rate tree's...
* drift
* distribution (Ho-Lee vs Black-Derman-Toy)
* nodes
* probabilities

## 1.2.

Consider an interest-rate cap with only a single caplet. Which of the following could we use as an underlying binomial tree to price this cap?

* interest-rate tree
* bond tree
* swap tree
* floor tree

## 1.3.

True or False: SABR is used to have structure in modeling implied volatilities across strikes, tenors, and expirations.

***

# **<span style="color:red">Solution </span>**

## **<span style="color:red">1.1 </span>**

* drift - term structure of spot rates, or more specifically, discount factors.
* distribution - assumption. We assumed log-normal (BDT) in the course but mentioned that we could easily substitute Ho-Lee.
* nodes - fit these to implied-volatilities fit to caps. These vols set the nodes, given that probabilities were set without loss of generality.
* probabilities - assumed to be .5, with nodes then restricted such that term structure and vols fit.

## **<span style="color:red">1.2 </span>**

Any of these will work as a one-period underlying tree for the cap. Anything with differential values across the two interest rates will work, as seen in Homework 1.

## **<span style="color:red">1.3 </span>**

False. SABR is used to model implied volatilities across strikes. The model implications are not used to understand volatilities across expirys or tenors--rather, SABR is fit to each expiry-tenor combination.

***

# 2. Pricing with BDT and Black

Use `rate curves` for market data.

## 2.1.

Recall that a floor is a portfolio of floorlets which
* depend on the realized interest rate one quarter before the expiration.
* each have the same strike interest rate.

Use Black's formula to price **just one floorlet**
* expiring at `T=3`
* struck at the `T=3` swap rate
* notional of `100`

## 2.2.

Use Black's formula to price the entire floor with expiration of `T=3`.

The floor has floorlets at `quarterly` frequency, except the first quarter.

## 2.3.

Use `rate tree` for a BDT binomial tree fit to market data for your convenience.

Use this binomial tree to price the floor described above.

Display the binomial tree of prices.

## 2.4.

What do you think is the most relevant reason for the difference in pricing between Black's formula and BDT?

## 2.5.

Use the BDT tree to price a swap...
* `receiving-fixed` 
* swap rate of `3.65%`
* tenor `T=3`
* notional `N=100`.
* frequency quarterly (`n=4`)

Display the pricing tree.

***

# **<span style="color:red">Solution </span>**

In [3]:
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)

from datetime import date
from datetime import datetime

import sys
sys.path.insert(0, '../cmds')
from ficcvol import *
from binomial import *
from ratecurves import *
from treasury_cmds import compound_rate
from volskew import *

In [4]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# %matplotlib inline
# plt.rcParams['figure.figsize'] = (12,6)
# plt.rcParams['font.size'] = 15
# plt.rcParams['legend.fontsize'] = 13

# import matplotlib.ticker as mtick
# from matplotlib.ticker import (MultipleLocator,
#                                FormatStrFormatter,
#                                AutoMinorLocator)

## **<span style="color:red">2.1. </span>**

In using Black's formula for the caplet, be careful to use 
* $T-dt$ for the time-to-expiration, as the uncertainty of the floorlet is resolved one period early.
* the **forward vol** as a given caplet is priced correctly with forward vol.

#### Note
Though the problem did not explicitly say the floorlet is quarterly (that is not said until `2.2`, it should be inferred given the curve data. Still, if one used a different frequency, that is fine. 

In [5]:
N = 100
Tfloor = 3
isPayer=False
strikefloor = curves['swap rates'][Tfloor]
freqfloor = 4

fwdrate = curves.loc[Tfloor,'forwards']
fwdvol = curves.loc[Tfloor,'fwd vols']

Z = curves.loc[Tfloor,'discounts']

In [6]:
floorlet_value = N * (1/freqfloor) * blacks_formula(Tfloor-1/freqfloor,fwdvol,strikefloor,fwdrate,Z,isCall=isPayer)
display(f'Floorlet value at T={Tfloor} is ${floorlet_value:.4f}.')

'Floorlet value at T=3 is $0.2749.'

## **<span style="color:red">2.2. </span>**

We use the same procedure as pricing the individual floorlet, but for all floorlets in the floor. Two considerations:

1. There is no floorlet expiring at $T=0.25$.
1. The **flat vol** at $T=3$ can be used in every floorlet to price the overall floor. Equivalently can use the forward vol at the expiration of each floorlet.

In [7]:
maturities = np.arange(1/freqfloor,Tfloor+1/freqfloor,1/freqfloor)
floorlets = pd.DataFrame(index=maturities,columns=['price'])

for i,Tval in enumerate(floorlets.index):
    if i==0:
        floorlets.loc[Tval] = 0
    else:
        floorlets.loc[Tval] = N * (1/freqfloor) * blacks_formula(Tval-1/freqfloor,curves.loc[Tval,'fwd vols'],strikefloor,curves.loc[Tval,'forwards'],curves.loc[Tval,'discounts'],isCall=isPayer)                        

In [8]:
floorlets.style.format('${:.6f}').format_index('{:.2f}')

,price
0.25,$0.000000
0.50,$0.000028
0.75,$0.006225
1.00,$0.033980
1.25,$0.141022
1.50,$0.183306
1.75,$0.216918
2.00,$0.239634
2.25,$0.272579
2.50,$0.273936


In [9]:
floor = floorlets.sum().to_frame().rename(columns={0:'floor'})
floor.style.format('${:.4f}')

,floor
price,$1.9148


## **<span style="color:red">2.3. </span>**

In [10]:
freqcurve = 4
dt = 1/freqcurve
compound = freqfloor
tsteps = int(Tfloor/dt)

In [11]:
refratetree = compound * (np.exp(ratetree / compound)-1)

if isPayer:
    payoff = lambda r: N * dt * np.maximum(r-strikefloor,0)
else:
    payoff = lambda r: N * dt * np.maximum(strikefloor-r,0)

cftree = payoff(refratetree.iloc[:tsteps,:tsteps])
### no caplet until second step, so ensure 0 until then
cftree.loc[0,0] = 0

#format_bintree(refratetree.iloc[:tsteps,:tsteps],style='{:.2%}')
#format_bintree(cftree)

In [12]:
floortree = bintree_pricing(payoff=payoff, ratetree=ratetree.iloc[:tsteps,:tsteps], undertree= refratetree.iloc[:tsteps,:tsteps], cftree=cftree, timing='deferred')
format_bintree(floortree,style='{:.2f}')

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75
state,,,,,,,,,,,,
0,1.60,1.02,0.58,0.27,0.10,0.03,0.00,0.00,0.00,0.00,0.00,0.00
1,,2.21,1.49,0.90,0.46,0.18,0.05,0.01,0.00,0.00,0.00,0.00
2,,,2.99,2.13,1.36,0.74,0.32,0.10,0.01,0.00,0.00,0.00
3,,,,3.88,2.94,2.00,1.17,0.55,0.18,0.02,0.00,0.00
4,,,,,4.51,3.51,2.55,1.68,0.93,0.34,0.05,0.00
5,,,,,,4.67,3.68,2.75,1.89,1.11,0.50,0.10
6,,,,,,,4.48,3.52,2.61,1.75,1.00,0.40
7,,,,,,,,4.04,3.09,2.18,1.34,0.61
8,,,,,,,,,3.42,2.47,1.57,0.74


## **<span style="color:red">2.4. </span>**

There are multiple reasons the prices differ. Answers here should focus on the biggest differences.

* The tree discretization: the dynamic of the quarterly floorlet is modeled in a single step in the tree.
* Related to the point above, the volatility operates differently in the tree than in Black's formula.

Other answers could be reasonable here.

## **<span style="color:red">2.5. </span>**

We can reuse...

* rate tree (continuously compounded) for discounting
* reference rate tree (quarterly compounded) for cashflows
* notional

In [13]:
strikeswap = .0365
isPayer = False

if isPayer:
    payoff = lambda r: N * dt * (r-strikeswap)
else:
    payoff = lambda r: N * dt * (strikeswap-r)

cftree = payoff(refratetree.iloc[:tsteps,:tsteps])

In [14]:
swaptree = bintree_pricing(payoff=payoff, ratetree=ratetree.iloc[:tsteps,:tsteps], undertree= refratetree.iloc[:tsteps,:tsteps], cftree=cftree, timing='deferred')
format_bintree(swaptree,style='{:.2f}')

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75
state,,,,,,,,,,,,
0,-1.26,-2.13,-2.92,-3.63,-4.25,-4.82,-5.22,-5.38,-5.23,-4.80,-3.88,-2.32
1,,0.36,-0.51,-1.32,-2.07,-2.81,-3.40,-3.79,-3.89,-3.73,-3.10,-1.90
2,,,1.76,0.89,0.06,-0.78,-1.52,-2.10,-2.45,-2.57,-2.28,-1.45
3,,,,2.86,2.00,1.11,0.29,-0.42,-0.97,-1.35,-1.40,-1.00
4,,,,,3.63,2.73,1.87,1.09,0.43,-0.12,-0.44,-0.43
5,,,,,,3.89,3.02,2.19,1.45,0.78,0.27,-0.01
6,,,,,,,3.81,2.96,2.16,1.41,0.78,0.29
7,,,,,,,,3.48,2.64,1.85,1.12,0.49
8,,,,,,,,,2.97,2.13,1.35,0.63


***

# 3. Callable Bond

#### Note:
Continue with the rate curves and BDT model provided in `rate curves` and `rate tree` in the provided spreadsheet.

## 3.1.

Consider a bond with:
* `T=3`
* face value of `N=100`
* coupons at `quarterly` frequency
* annualized coupon rate of `cpn=6%`.

Use your BDT tree to price this bond and display the pricing tree.

## 3.2. (7pts)

Suppose the bond is callable by the issuer.

* `European` style
* expiration of `Topt=1.5`
* (clean) `strike=100`

What is the value of the issuer's call option?

*Note that there is no difference between clean or dirty strike given that the bond pays coupons quarterly, and we are looking at quarterly steps in the tree.*

## 3.3. (3pts)

What is the value of the callable bond?

## 3.4.

We found that the Freddie Mac callable bonds often have negative option-adjusted spreads (OAS). 

Why was this? Do you expect that the european callable bond would be less prone to this phenomenon?

## 3.5.

We found that the Freddie Mac american callable bond never priced above 100.

Does this bond ever price above 100? Explain why this is possible when it was not for the Freddie Mac callable.

## 3.6.

Price the callable bond without using binomial trees.

* Use standard closed-form pricing for the vanilla bond, given the rate curve data.
* Use Black's formula to price the callable option.

Report this newly modeled price.

*Note: In Black's formula, use the flat volatility for the option term.*

## 3.7.

Which assumptions of Black's formula do we prefer to Black-Scholes for this problem?

## 3.8.

Which aspects of the calculation in `3.6` differ because we are using Black's formula, not Black-Scholes? 

*Note: Unlike `3.7`, we're asking about the differences of implementing the calculation, not the differences in the assumptions of the models.

## 3.9. 

Suppose that we buy the callable bond, but we want to hedge against it being called by the issuer. That is to say, we want to retain upside exposure to rates decreasing while otherwise keeping the nature of the position the same.

Explain specifically how you would use caps, floors, or swaptions to achieve this. 

No need to calculate the value; rather, just describe the specific product you would go long (or short) and how it would transform your exposure.

## 3.10.

How would your answer to `3.9` change if it were a callable bond with **american** exercise by the issuer?

***

# **<span style="color:red">Solution </span>**

## **<span style="color:red">3.1 </span>**

In [15]:
T=3
FACE=100
cpn_freq = 4
cpn = .06

In [16]:
wrapper_bond = lambda r: payoff_bond(r, dt, facevalue=FACE * (1+cpn/cpn_freq))
payoff_call = lambda p: np.maximum(p-STRIKE,0)

In [17]:
cftree = construct_bond_cftree(T, compound, cpn, cpn_freq)

In [18]:
def highlight_values(val,thresh=100):
    if pd.isna(val):
        color = '#d3d3d3'  # Light grey for NaN values
    elif val < thresh:
        color = '#ffcccc'  # Light red
    else:
        color = '#ccffcc'  # Light green
    return f'background-color: {color}'

In [19]:
bondtree = bintree_pricing(payoff=wrapper_bond, ratetree=ratetree, cftree=cftree)
bondtree.style.format('{:.2f}',na_rep='').applymap(highlight_values).format_index('{:.2f}',axis=1)

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75
state,,,,,,,,,,,,
0,105.31,103.90,102.57,101.32,100.16,99.04,98.10,97.39,96.99,96.88,97.25,98.25
1,,106.46,105.05,103.69,102.39,101.10,99.95,99.01,98.35,97.96,98.03,98.67
2,,,107.38,105.96,104.57,103.17,101.87,100.73,99.81,99.13,98.86,99.12
3,,,,107.98,106.55,105.10,103.71,102.43,101.31,100.37,99.75,99.58
4,,,,,108.23,106.75,105.32,103.97,102.73,101.61,100.72,100.15
5,,,,,,107.94,106.49,105.09,103.76,102.52,101.43,100.57
6,,,,,,,107.30,105.87,104.49,103.16,101.94,100.87
7,,,,,,,,106.40,104.98,103.60,102.29,101.08
8,,,,,,,,,105.31,103.89,102.52,101.22


In [20]:
accint = construct_accint(bondtree.columns.values, compound, cpn)
cleantree = np.maximum(bondtree.subtract(accint,axis=1),0)

cleantree.style.format('{:.2f}',na_rep='').applymap(highlight_values).format_index('{:.2f}',axis=1)

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75
state,,,,,,,,,,,,
0,105.31,102.40,102.57,99.82,100.16,97.54,98.10,95.89,96.99,95.38,97.25,96.75
1,,104.96,105.05,102.19,102.39,99.60,99.95,97.51,98.35,96.46,98.03,97.17
2,,,107.38,104.46,104.57,101.67,101.87,99.23,99.81,97.63,98.86,97.62
3,,,,106.48,106.55,103.60,103.71,100.93,101.31,98.87,99.75,98.08
4,,,,,108.23,105.25,105.32,102.47,102.73,100.11,100.72,98.65
5,,,,,,106.44,106.49,103.59,103.76,101.02,101.43,99.07
6,,,,,,,107.30,104.37,104.49,101.66,101.94,99.37
7,,,,,,,,104.90,104.98,102.10,102.29,99.58
8,,,,,,,,,105.31,102.39,102.52,99.72


## **<span style="color:red">3.2 </span>**

In [21]:
Topt = 1.5
STRIKE = 100
CLEANCALL = True

In [22]:
tsteps = int(Topt/dt)

if CLEANCALL:
    undertree = cleantree
else:
    undertree = bondtree
    
calltree = bintree_pricing(payoff=payoff_call, ratetree=ratetree.iloc[:tsteps+1,:tsteps+1], undertree= undertree.iloc[:tsteps+1,:tsteps+1])
format_bintree(calltree)

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50
state,,,,,,,
0,3.34,2.62,1.87,1.12,0.45,0.00,0.00
1,,4.15,3.45,2.67,1.82,0.92,0.00
2,,,4.95,4.31,3.58,2.76,1.87
3,,,,5.68,5.13,4.48,3.71
4,,,,,6.33,5.87,5.32
5,,,,,,6.87,6.49
6,,,,,,,7.30


## **<span style="color:red">3.3 </span>**

In [23]:
callabletree_clean = cleantree - calltree
callabletree_dirty = bondtree - calltree

format_bintree(callabletree_clean)

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75
state,,,,,,,,,,,,
0,101.97,99.78,100.71,98.70,99.71,97.54,98.10,,,,,
1,,100.81,101.60,99.52,100.57,98.67,99.95,,,,,
2,,,102.44,100.15,100.99,98.91,100.00,,,,,
3,,,,100.80,101.42,99.12,100.00,,,,,
4,,,,,101.89,99.38,100.00,,,,,
5,,,,,,99.58,100.00,,,,,
6,,,,,,,100.00,,,,,
7,,,,,,,,,,,,
8,,,,,,,,,,,,


In [24]:
format_bintree(callabletree_dirty)

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75
state,,,,,,,,,,,,
0,101.97,101.28,100.71,100.20,99.71,99.04,98.10,,,,,
1,,102.31,101.60,101.02,100.57,100.17,99.95,,,,,
2,,,102.44,101.65,100.99,100.41,100.00,,,,,
3,,,,102.30,101.42,100.62,100.00,,,,,
4,,,,,101.89,100.88,100.00,,,,,
5,,,,,,101.08,100.00,,,,,
6,,,,,,,100.00,,,,,
7,,,,,,,,,,,,
8,,,,,,,,,,,,


## **<span style="color:red">3.4 </span>**

There are multiple potential causes to the underpricing of callable Freddie Mac bonds via our models.

But we focused most on the idea that those bonds were **american** style, and our model assumed optimal exercise.

In reality, research shows the issuer (Freddie Mac) does not exercise optimally, which means the embedded short call option is not as costly to investors as it would seem to be, leaving the market price higher.

Given that this is **european**, the exercise decision is less complicated. Thus, we might expect less of an increase in market value due to the issuer's suboptimal exercise, meaning we'd expect the negative OAS to be moderated.

## **<span style="color:red">3.5 </span>**

Yes, this bond prices over 100 in most nodes, including the present value.

The Freddie Mac bond was American, so any excess value could be taken by the issuer via calling the bond. Here, any excess value up until the option expiration accrues to the investor, (via the larger-than-market-value coupons.)

## **<span style="color:red">3.6 </span>**

For the price of the vanilla bond, just use the present-value formula. 

In [25]:
from bondmath import bond_pricer_formula
px_vanilla = bond_pricer_formula(T,curves.loc[T,'spot rates'],cpn=cpn,freq=cpn_freq)

pd.DataFrame([px_vanilla,cleantree.iloc[0,0]],index=['bond formula','tree'],columns=['clean price']).style.format('${:.2f}')

,clean price
bond formula,$105.39
tree,$105.31


### Complication: Forward Price of Bond
For Black's formula, we need the *forward* bond price. 

From FINM 37400, this is straightforward to calculate, though it requires a few assumptions. 

#### Details
To get the forward price, we assume
* the market price of treasuries matches the price we model with the closed-form solution for vanilla bonds (rather than the tree).
* the bond with maturity at the option term has a coupon rate equal to that of the bond in which we are interested.

Procedure:
* calculate the formulaic (assumed market) price of bonds with maturities at the option term and the vanilla bond term.
* calculate them as both having the same coupon
* via shorting the bond with maturity equal to the option, we can mimic a forward contract on the bond
* this requires the forward ratio to establish the long-short position.
* we are designing it such that the coupons offset, so the forward price is just the ratio of these prices.

$$P_{\text{forward}}(T_\text{option}\to T) = 100\frac{P(T)}{P(T_\text{option})}$$

where $P(\tau)$ denotes the price of a bond with maturity at $\tau$.

In [26]:
px_vanilla_Topt = bond_pricer_formula(Topt,curves.loc[Topt,'spot rates'],cpn=cpn,freq=cpn_freq)
px_fwd = FACE * px_vanilla / px_vanilla_Topt

pd.DataFrame([px_vanilla,px_vanilla_Topt,px_fwd],index=[f'maturity {T:.1f}',f'maturity {Topt:.1f}',f'forward from {Topt:.1f} to {T:.1f}'],columns=['price']).style.format('${:.2f}')

,price
maturity 3.0,$105.39
maturity 1.5,$102.02
forward from 1.5 to 3.0,$103.31


### Complication: Bond volatility

The problem already simplified by having you use the flat volatility.

However, this is the volatility of interest rates, not bond prices.

From FINM 37400, we know that up to a linear approximation, the vol of rates will scale to the vol of bonds via the duration of the bond.

Thus, we can approximate...

$$\sigma_{\text{bond}} \approx D \times \sigma_{\text{rate}}\times r$$

where again, we're abstracting from forward vol and forward price vol.

In [27]:
from bondmath import duration_closed_formula

duration = duration_closed_formula(T,curves.loc[T,'spot rates'],cpn,freq=cpn_freq)
vol_bond = duration * curves.loc[Topt,'flat vols'] * curves.loc[T,'forwards']

pd.DataFrame([curves.loc[Topt,'flat vols'], duration, vol_bond],index=['rate vol (flat)','bond duration','bond vol'],columns=['estimates']).style.format('{:.2%}')

,estimates
rate vol (flat),27.29%
bond duration,277.52%
bond vol,2.68%


#### Putting it together

In [28]:
px_calloption = blacks_formula(Topt,vol_bond,STRIKE,px_fwd,discount=curves.loc[Topt,'discounts'])
px_callable_bond = px_vanilla - px_calloption

tab = pd.DataFrame([px_vanilla,px_calloption,px_callable_bond],index=['vanilla bond','call option','callable bond'],columns=['formulaic prices'])
tab['tree'] = [bondtree.iloc[0,0], calltree.iloc[0,0],callabletree_dirty.iloc[0,0]]
tab.style.format('{:.2f}')

,formulaic prices,tree
vanilla bond,105.39,105.31
call option,3.35,3.34
callable bond,102.04,101.97


## **<span style="color:red">3.7 </span>**

We prefer that Black's formula models the forward price, as we know the bond's spot price can't follow the geometric brownian motion of Black-Scholes. (As discussed in Week 1, bond prices have autocorrelation and other issues.)

More obviously, we prefer that Black's formula allows for time-varying interest rates.

## **<span style="color:red">3.8 </span>**

Given the assumptions above, using Black's formula requires that as inputs we use...

* the forward, not the spot. (In this example forward price, but could be a forward rate for a swaption.)
* the discount factor, not $e^{-rT_{\text{opt}}}$

## **<span style="color:red">3.9 </span>**

There are a few ways to construct this.

* receiver swaption or long a floor are the obvious approaches.

1. 1.5y $\rightarrow$ 1.5y receiver-swaption, with strike of 6%
    * If bond is called, you use the money received for the bond (the strike) to invest in a new par bond, which will pay a coupon at the prevailing interest rate. Simultaneously exercise the swaption to swap a floating rate (hopefully close to your new bond's coupon) and instead receive the 6% that you were originally getting.
    * Benefit here is that you retain the 6% rate, and you don't pay for more optionality than is needed. You are subject to a single call date, and you have a single option to offset that.
    * Only problem is that the floating rate paid in the swaption may not equal the coupon being received on the new bond.

2. Buying a floor
    * Similar to above, but you have optionality at each payment date, which is more optionality than you need given that the bond will or will not be called at one particular date. 
    * Thus, may be paying for more optionality than is needed.

3. Buying a cap or selling a floor
    * This wouldn't make sense, as it wouldn't offset the option exposure which you are short.

## **<span style="color:red">3.10 </span>**

1. Make the answer above an American swaption. But these are not typical.

2. With the bond being an American callable bond, now it is useful to have the additional opitonality in buying the floor described above. Still not a great match in that you pay for optionality beyond what is needed.

***

# 4. Swaptions and SABR

## 4.1.

Use the market data in `rate curves` to calculate the relevant forward swap rate for an at-the-money (ATM) swaption with...
* `expiration=1`
* `tenor=2`

Report this forward swap rate.

## 4.2.

#### Note

Regardless of what you calculated in the previous problem **use a forward swap rate of `.0365` for the rest of this section**, not just this specific question.

Don't worry that the forward swap rate provided here may not match your answer from the previous section. We are using it to ensure all solutions below are based on the same forward swap rate.

#### Continuing...

Use the data in `volskew` which gives market quotes (in terms of Black vol) on swaptions across various strikes. 
* The strikes listed are relative to the ATM strike, which equals the forward swap rate given to you in the previous paragraph.
* All these quotes are for the same expiration and tenor considered in `4.1`.

Report the price of the swaption with specification of...
* struck at the money
* notional of `100`
* a `receiver` swaption

## 4.3. (10pts)

Use SABR to consider pricing for strikes which are not listed. Sepcifically,

* Input the SABR parameters given in `sabrparams` in the exam data sheet to get the implied volatility for any strike.

* To do this, choose the `SLIM` SABR model, where $\alpha$ (also called $\sigma_0$) is a function of the other SABR parameters. Thus, you won't make use of the estimated $\alpha$ parameter; rather, you'll make use of the other parameters in conjunction with the ATM market quoted vol.

Consider a `STRIKE=.03`.

Report
* the SABR-implied vol for this strike, (and the provided forward swap rate.) 
* the price for this swaption given by Black's formula.

## 4.4. 

Suppose the forward swap rate changes by `+10bps`.

(We are considering an instantaneous change, so no need to change the time-to-expiration.)

Report
* the new price, assuming vol stays constant
* the new vol and the new price, assuming vol responds according to the SABR specification of part `4.3`.

## 4.5.

Use these new prices to calculate and report the approximate delta, for both a static and dynamic vol.

Specifically, calculate the numerical change in price per change in forward rate when...
* holding vol constant
* modeling vol changing according to SABR

How much does the "augmented" delta differ from the classic delta?

## 4.6.

Without doing any new calculation, what do you think the effect of a SABR model would be on delta for a payer-swaption in response to a decrease in interest rates?

* Would the delta be positive or negative?
* Would the "augmented" effects of SABR cause the price to be larger or smaller than what the classic delta would imply? 

## 4.7.

Consider again the given forward swap rate provided in `4.2` (without the shift considered in part `4.4`.)

Use the SABR vol skew to price the swaption (same expiry and tenor) struck `-300bps` OTM. 

* Report this model price and compare it to the market quote at -300bps.

* Why might we trust the model price more than the provided market quote here?

## 4.8.

What advantage does SABR have over local vol models? 

Be specific.

## 4.9.

For the estimated SABR model in the picture, is ATM implied volatility higher or lower as the interest rate goes higher?

In a sentence, describe specifically what vol path indicates to us about this option market.

(If the figure is not rendering in this cell, find it in `../data/volpath_example.png`.

![title](../data/volpath_example.png)

## 4.10.

How do we quantify the **vol path** in SABR? Is it estimated or assumed? Be specific.

***

# **<span style="color:red">Solution </span>**

## **<span style="color:red">4.1 </span>**

In [29]:
SWAP_TYPE = 'SOFR'
QUOTE_STYLE = 'black'
RELATIVE_STRIKE = 0

expry = 1
tenor = 2

In [30]:
freqswap = 4

Topt = expry
Tswap = Topt+tenor

fwdswap_calc = calc_fwdswaprate(curves['discounts'], Topt, Tswap, freqswap=freqswap)

In [31]:
pd.DataFrame([fwdswap_calc],index=[f'forward swap rate {Topt:.1f} to {Tswap:.1f}'],columns=['']).style.format('{:.4%}')

,
forward swap rate 1.0 to 3.0,3.6651%


## **<span style="color:red">4.2 </span>**

Given that we are calculating the ATM swaption, it doesn't matter that it is the **receiver** swaption vs the **payer** swaption.

In [32]:
F = .0365
isPayer = False
N = 100

# asking ATM
SKEW = 0
volATM = volskew.loc[0,SKEW]/100

Get the sum of discount factors, noting the spacing.

In [33]:
period_fwd = curves.index.get_loc(Topt)
period_swap = curves.index.get_loc(Tswap)+1
step = round(freqcurve/freqswap)

discount_swaption = curves['discounts'].iloc[period_fwd+step : period_swap : step].sum()/freqswap

Then by Black,

In [34]:
px_swaptionATM = N * blacks_formula(Topt,volATM,F,F,discount_swaption,isCall=isPayer)

tab_swaption = pd.DataFrame([px_swaptionATM],index=['ATM'],columns = ['price'])
tab_swaption.style.format('${:.4f}')

,price
ATM,$0.9209


## **<span style="color:red">4.3 </span>**

In [35]:
STRIKE = .03
doSLIM = True

# unpack the parameters in the data
beta,alpha,nu,rho = sabrparams.iloc[:,0]

volOTM = sabr_slim(beta,nu,rho,F,STRIKE,Topt,volATM)

px_swaptionOTM = N * blacks_formula(Topt,volOTM,STRIKE,F,discount_swaption,isCall=isPayer)

In [36]:
tab_swaption.loc['OTM'] = px_swaptionOTM
tab_swaption['vol'] = [volATM,volOTM]
tab_swaption['strike'] = [F,STRIKE]
tab_swaption['forward'] = F
tab_swaption.style.format({'vol':'{:.2%}','price':'{:.4f}','strike':'{:.2%}','forward':'{:.2%}'})

,price,vol,strike,forward
ATM,0.9209,34.80%,3.65%,3.65%
OTM,0.4662,39.31%,3.00%,3.65%


## **<span style="color:red">4.4 </span>**

The problem intends for you to recompute the OTM swaption price given the shift in rates.

This is considered as an instant shift, so there is no need to adjust time-to-expiration and time-to-maturity.

In [37]:
SHIFT = 10/100/100

volOTMshift = sabr_slim(beta,nu,rho,F+SHIFT,STRIKE,Topt,volATM)
px_swaptionOTMshift_static = N * blacks_formula(Topt,volOTM,STRIKE,F+SHIFT,discount_swaption,isCall=isPayer)
px_swaptionOTMshift_dynamic = N * blacks_formula(Topt,volOTMshift,STRIKE,F+SHIFT,discount_swaption,isCall=isPayer)

tab_swaption.loc['OTM shift static'] = [px_swaptionOTMshift_static, volOTM, STRIKE, F+SHIFT]
tab_swaption.loc['OTM shift dynamic'] = [px_swaptionOTMshift_dynamic, volOTMshift, STRIKE, F+SHIFT]

tab_swaption.style.format({'vol':'{:.2%}','price':'{:.4f}','strike':'{:.2%}','forward':'{:.2%}'})

,price,vol,strike,forward
ATM,0.9209,34.80%,3.65%,3.65%
OTM,0.4662,39.31%,3.00%,3.65%
OTM shift static,0.4237,39.31%,3.00%,3.75%
OTM shift dynamic,0.4385,40.04%,3.00%,3.75%


We see that given a shift in rates, the swaption price drops less due to a rise in vol.

## **<span style="color:red">4.5 </span>**

In [38]:
tab_delta = tab_swaption[['price']].drop(index=['ATM']).copy()
tab_delta['diff'] = tab_delta - tab_swaption.loc['OTM','price']
tab_delta['delta'] = tab_delta['diff'] / SHIFT
tab_delta.style.format({'diff':'{:.4f}','delta':'{:.2f}','price':'{:.4f}'})

,price,diff,delta
OTM,0.4662,0.0000,0.00
OTM shift static,0.4237,-0.0425,-42.51
OTM shift dynamic,0.4385,-0.0277,-27.70


## **<span style="color:red">4.6 </span>**

For **payer** swaption, the delta is positive.

Given the **decreased** rate, the swaption is less in-the-money, and price goes down.

As for the SABR vol effect, it could go either way. 
* Possible SABR implies decrease rate will once again increase vol (as did the increased rate) above. This will increase the price, which will attenuate the positive delta to be a smaller "augmented" delta.

* Also possible SABR implies decrease in rate will lower vol, further lowering swaption price. This would mean the "augmented" delta is even more positive than the classic delta.

Fine if assumed SABR would increase or decrease vol (no way to know without calculating). Graded for logic of that implication.

## **<span style="color:red">4.7 </span>**

In [39]:
SWAP_TYPE = 'SOFR'
QUOTE_STYLE = 'black'
RELATIVE_STRIKE = 0

expry = 1
tenor = 2

freqswap = 4

Topt = expry
Tswap = Topt+tenor

F = .0365
isPayer = False
N = 100

# asking ATM
SKEW = -300
volATM = volskew.loc[0,SKEW]/100

period_fwd = curves.index.get_loc(Topt)
period_swap = curves.index.get_loc(Tswap)+1
step = round(freqcurve/freqswap)

discount_swaption = curves['discounts'].iloc[period_fwd+step : period_swap : step].sum()/freqswap

px_swaption = N * blacks_formula(Topt,volATM,F + SKEW/100/100,F,discount_swaption,isCall=isPayer)

tab_swaption_market = pd.DataFrame([px_swaption],index=['-300'],columns = ['Market Price'])
tab_swaption_market.style.format('${:.4f}')

,Market Price
-300,$0.0448


In [40]:
STRIKE = F + SKEW/100/100
volATM = volskew.loc[0,0]/100
volOTM = sabr_slim(beta,nu,rho,F,STRIKE,Topt,volATM)
px_swaptionOTM = N * blacks_formula(Topt,volOTM,STRIKE,F,discount_swaption,isCall=isPayer)
tab_swaption_SABR = pd.DataFrame([px_swaptionOTM],index=['-300'],columns = ['SABR Price'])
tab_swaption_SABR.style.format('${:.4f}')

,SABR Price
-300,$0.0231


We can trust the SABR model more because market dynamics and liquidity issues may change the price of a security drastically compared to it's modeled fair value. In this example, a deep OTM option would likely be illiquid, causing it's market price to deviate from the modeled fair value. 

## **<span style="color:red">4.8 </span>**

Local vol models can perfectly fit the quoted options while still modeling the skew. However, they may be **overfitting** and do worse out of sample.

SABR and other stochastic vol models retain no-arbitrage implications (unlike splines, etc.) while giving structure that may be useful in modeling other empirical facts, such as the vol path.

## **<span style="color:red">4.9 </span>**

Be careful that you do not answer about the **strike** increasing, which is the x-axis.

Then, the answer hinges on which we are considering...
* keeping strike ATM, we are looking along the vol path, and the implied vol is getting lower.
* keeping strike fixed, we are looking at a vertical slice of the figure. We see 
    * vol goes up with rate if strike is low
    * vol goes down with rate if strike is high.

Again, any response consistent with these facts is fine. No need to explain all contingencies.

## **<span style="color:red">4.10 </span>**

The vol paths are estimated using the ATM strike implied volatilities of the given options. 

***